# RouterChain

## 提示词

In [2]:
from langchain.prompts import PromptTemplate

# 物理
physics_template = """您是一位聪明的物理教授,\n
您擅长以简洁易懂的方式回答物理问题。\n
当您不知道问题答案时，您会坦诚承认自己不知道。\n
下面是一个问题：
{input_question}
"""
physics_prompt = PromptTemplate.from_template(physics_template)

# 数学
math_template = """您是一位聪明的数学教授,\n
您擅长回答数学问题。\n
您之所以如此优秀，是应为您在回答问题时，会将问题拆分为多个组分，\n
回答这些个组分，然后将他们组合起来，回答更加广泛的问题。\n
下面是一个问题：
{input_question}
"""
math_prompt = PromptTemplate.from_template(math_template)

## Chains

In [4]:
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain_ollama.llms import OllamaLLM

## 路由表
prompt_infos = [
  {
    "name":"physics",
    "description":"擅长回答物理问题",
    "prompt_template":physics_template
  },
  {
    "name":"math",
    "description":"擅长回答数学问题",
    "prompt_template":math_template
  }
]

llm = OllamaLLM(
  model="llama3.1:8b",
  temperature=0
)

destination_chains = {}

# 循环遍历路由表
for p_info in prompt_infos:
  name = p_info["name"]
  prompt_template= p_info["prompt_template"]
  prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['input']
  )

  chain = LLMChain(
    llm= llm,
    prompt= prompt,
  )
  destination_chains[name] = chain

# 未命中的对话链
default_chain = ConversationChain(
  llm= llm,
  output_key= "text",
)

/var/folders/d6/8jzk9xqn419djvdnx23ft3y40000gn/T/ipykernel_75006/4023973554.py:35: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
/var/folders/d6/8jzk9xqn419djvdnx23ft3y40000gn/T/ipykernel_75006/4023973554.py:42: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(


## **RouterChain**

In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains.router import RouterChain, MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import LLMRouterChain

# 构建 destinations 字符串
destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# 格式化路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
print(router_template)

# 定义路由器的 PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    # output_parser=
)

# 构建 LLMRouterChain
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt,
)


chain = MultiPromptChain(
  router_chain=router_chain,
  destination_chains=destination_chains,
  default_chain=default_chain,
  verbose=True
)

# 示例输入
input_text = "请告诉我关于二战的历史。"
parsed_output = chain.run(input_text)
print(parsed_output)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics:擅长回答物理问题
math:擅长回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of 

ValidationError: 1 validation error for LLMRouterChain
  Value error, LLMRouterChain requires base llm_chain prompt to have an output parser that converts LLM text output to a dictionary with keys 'destination' and 'next_inputs'. Received a prompt with no output parser. [type=value_error, input_value={'llm_chain': LLMChain(ve...arser(), llm_kwargs={})}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error